In [5]:
!pip install pykrx

In [6]:
!pip install pandas

In [7]:
import os
import openai
from pykrx import stock
import pandas as pd
from datetime import datetime
import json
from difflib import get_close_matches
from io import StringIO

# ✅ OpenAI API 키 가져오기
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("❌ OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

# ✅ 사용자 입력
user_input = input("분석할 종목명과 기간을 입력하세요 (예: 삼성전자 2023년 1월 1일부터 2023년 6월 30일까지 분석해줘): ")

# ✅ GPT에게 종목명, 날짜 추출 요청
extract_response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "사용자 문장에서 종목명, 시작일, 종료일을 추출해서 JSON 문자열로 반환하세요. 예: {'종목명':'삼성전자', '시작일':'2023-01-01', '종료일':'2023-06-30'}"},
        {"role": "user", "content": user_input}
    ],
    temperature=0.1
)

# ✅ JSON 파싱
extracted_text = extract_response.choices[0].message.content
extracted_data = json.loads(extracted_text.replace("'", '"'))

stock_name = extracted_data["종목명"]
start_date = extracted_data["시작일"].replace("-", "")
end_date = extracted_data["종료일"].replace("-", "")

print(f"\n✅ 입력 종목명: {stock_name}, 기간: {start_date} ~ {end_date}")

# ✅ 전체 종목 이름/코드 리스트 불러오기
tickers = stock.get_market_ticker_list()
name_to_code = {stock.get_market_ticker_name(code): code for code in tickers}

# ✅ 정확히 일치하지 않으면 유사한 종목 추천
def find_best_match(name, all_names):
    if name in all_names:
        return name
    matches = get_close_matches(name, all_names, n=5, cutoff=0.5)
    if not matches:
        print(f"❌ '{name}'와 유사한 종목명을 찾을 수 없습니다.")
        exit()
    print(f"❓ 정확한 종목명을 찾을 수 없습니다. 다음 중 선택하세요:")
    for i, match in enumerate(matches):
        print(f"{i+1}. {match}")
    choice = input("번호를 입력하세요 (예: 1): ")
    try:
        return matches[int(choice)-1]
    except:
        print("❌ 잘못된 입력입니다.")
        exit()

matched_name = find_best_match(stock_name, list(name_to_code.keys()))
ticker = name_to_code[matched_name]

print(f"✅ 선택된 종목: {matched_name}, 코드: {ticker}")

# ✅ 주가 데이터 가져오기
df = stock.get_market_ohlcv(start_date, end_date, ticker)

if df.empty:
    print("❌ 선택한 기간에 대한 주가 데이터가 없습니다.")
    exit()

# ✅ GPT에 전달할 CSV 데이터
csv_buffer = StringIO()
df.to_csv(csv_buffer)
csv_data = csv_buffer.getvalue()

# ✅ GPT에게 주가 분석 요청
analysis_response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"다음은 {matched_name}의 날짜별 주가 데이터입니다. 이 데이터를 분석해서 기간 내 특징을 요약 보고해주세요."},
        {"role": "user", "content": csv_data}
    ],
    temperature=0.5,
    max_tokens=1024
)

# ✅ 분석 결과 출력
print("\n📊 GPT 분석 결과:")
print(analysis_response.choices[0].message.content)


ValueError: ❌ OPENAI_API_KEY 환경변수가 설정되지 않았습니다.

Bad pipe message: %s [b'"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";']
Bad pipe message: %s [b'"138"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Wi', b'ows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36\r\nAccept: text/']
Bad pipe message: %s [b'ml,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/s']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: "Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"\r\nsec-ch-ua-mobile: ?0\r\ns']
Bad pipe message: %s [b'-ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; ', b'n64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36\r\nAccep', b' text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signe']


In [ ]:
df